In [1]:
import pandas as pd
import os
import glob
import plotly.graph_objects as go
from utils import util 
from plotly.subplots import make_subplots

In [2]:
def get_each_video_emo_pos(run_name, video_name):
    root_dir = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/'
    target_dir = root_dir + run_name
    fig = go.Figure()
    df = pd.DataFrame()
    
    csv_path_list = glob.glob(os.path.join(target_dir, "**/", f"*_pred.csv"), recursive=True)
    for csv_path in csv_path_list:
        _df = pd.read_csv(csv_path)
        df = pd.concat([df, _df])
    df = df.reset_index(drop=True)
    
    # extract video name and frame number from img_path
    # use util.get_video_name_and_frame_num
    df['video_name'], df['frame_num'] = zip(*df['img_path'].map(util.get_video_name_and_frame_num))
    
    video_df = df[df['video_name'] == video_name]
    video_df = video_df.reset_index(drop=True)
    video_df = video_df.sort_values('frame_num')
    
    # plot
    emo = 0
    for i in range(len(video_df)):
        _emo = video_df['emo_gt'][i]
        if emo != _emo:
            if _emo == 0:
                # write vertical line
                fig.add_trace(go.Scatter(x=[video_df['frame_num'][i]]*2, y=[0, 1], mode='lines', line=dict(color='blue', width=1, dash='dash')))
            elif _emo == 1:
                fig.add_trace(go.Scatter(x=[video_df['frame_num'][i]]*2, y=[0, 1], mode='lines', line=dict(color='red', width=1, dash='dash')))
            elif _emo == 2:
                fig.add_trace(go.Scatter(x=[video_df['frame_num'][i]]*2, y=[0, 1], mode='lines', line=dict(color='green', width=1, dash='dash')))
            emo = _emo
            
    fig.add_trace(go.Scatter(x=video_df['frame_num'], y=video_df['emo_pos'], mode='markers', marker=dict(color='orange')))
    # horizontal line at 0.5
    fig.add_trace(go.Scatter(x=video_df['frame_num'], y=[0.5]*len(video_df), mode='lines', name='0.5', line=dict(color='black', width=1, dash='dash')))
    fig.update_layout(title=f'{run_name} <{video_name}> posteriors', xaxis_title='frame', yaxis_title='posteriors')
    # no legend
    fig.update_layout(showlegend=False)
    # tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))
    fig.show()

In [3]:
def compare_emo_pos(run_name_list, video_name, legend_list=None):
    root_dir = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/'
    fig = go.Figure()
    emo_pos_df = pd.DataFrame()
    for i, run_name in enumerate(run_name_list):
        target_dir = root_dir + run_name
        df = pd.DataFrame()
       
        csv_path_list = glob.glob(os.path.join(target_dir, "**/", f"*_pred.csv"), recursive=True)
        for csv_path in csv_path_list:
            _df = pd.read_csv(csv_path)
            df = pd.concat([df, _df])
        df = df.reset_index(drop=True)
        df['video_name'], df['frame_num'] = zip(*df['img_path'].map(util.get_video_name_and_frame_num))
        
        video_df = df[df['video_name'] == video_name]
        video_df = video_df.reset_index(drop=True)
        video_df = video_df.sort_values('frame_num')
        
        emo_pos_df['img_path'] = video_df['img_path']
        emo_pos_df['frame_num'] = video_df['frame_num']
        emo_pos_df[run_name] = video_df['emo_pos']
        
        # plot
        if legend_list is not None:
            fig.add_trace(go.Scatter(x=video_df['frame_num'], y=video_df['emo_pos'], mode='markers', name=legend_list[i]))
        else:
            # color is random
            fig.add_trace(go.Scatter(x=video_df['frame_num'], y=video_df['emo_pos'], mode='markers', name=run_name))
    
    # draw emo_pos average
    emo_pos_df['mean'] = emo_pos_df.iloc[:, 2:].mean(axis=1)
    fig.add_trace(go.Scatter(x=emo_pos_df['frame_num'], y=emo_pos_df['mean'], mode='markers', name='average'))
    
    emo = 0
    for i in video_df['frame_num']:
        _emo = video_df[video_df['frame_num'] == i]['emo_gt'].values[0]
        if emo != _emo:
            # write vertical line
            if _emo == 0:
                fig.add_trace(go.Scatter(x=[i]*2, y=[0, 1], mode='lines', line=dict(color='blue', width=1, dash='dash'), showlegend=False))
            elif _emo == 1:
                fig.add_trace(go.Scatter(x=[i]*2, y=[0, 1], mode='lines', line=dict(color='red', width=1, dash='dash'), showlegend=False))
            elif _emo == 2:
                fig.add_trace(go.Scatter(x=[i]*2, y=[0, 1], mode='lines', line=dict(color='green', width=1, dash='dash'), showlegend=False))
        emo = _emo
    
    # horizontal line at 0.5
    fig.add_trace(go.Scatter(x=video_df['frame_num'], y=[0.5]*len(video_df), mode='lines', name='0.5', line=dict(color='black', width=1, dash='dash'), showlegend=False))
    fig.update_layout(title=f'<{video_name}> posteriors', xaxis_title='frame', yaxis_title='posteriors')
    
    fig.update_yaxes(range=[0, 1])
    
    # tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))
    fig.show()
    
    return emo_pos_df     

In [4]:
def compare_emo_pos_all_video(run_name_list, video_name_list, legend_list=None, figure_size=(1000, 1000), show_average=False, epoch=None):
    size = 0
    while True:
        size += 1
        if size**2 >= len(video_name_list):
            break
    
    color_list = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
    root_dir = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/'
    base_df = pd.read_csv('/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/emo_and_au-gaze-hp(video1-25).csv')
    base_df['video_name'], base_df['frame_num'] = zip(*base_df['img_path'].map(util.get_video_name_and_frame_num))
    col_len = size
    row_len = size
    if len(video_name_list) % size == 0:
        row_len -= 1
    fig = make_subplots(rows=row_len, cols=col_len, subplot_titles=video_name_list)
    # change figure size
    fig.update_layout(width=figure_size[0], height=figure_size[1])
    
    row = 1
    col = 1
    for video_name in video_name_list:
        if col > size:
            row += 1
            col = 1
        emo_pos_df = pd.DataFrame()
        for i, run_name in enumerate(run_name_list):
            if epoch is not None:
                target_dir = root_dir + run_name + f'/epoch{epoch[i]}/'
            else:
                target_dir = root_dir + run_name
            color = color_list[i % len(color_list)]
            df = pd.DataFrame()
            
            csv_path_list = glob.glob(os.path.join(target_dir, "**/", f"*_pred.csv"), recursive=True)
            for csv_path in csv_path_list:
                _df = pd.read_csv(csv_path)
                df = pd.concat([df, _df])
            df = df.reset_index(drop=True)
            df['video_name'], df['frame_num'] = zip(*df['img_path'].map(util.get_video_name_and_frame_num))
            
            video_df = df[df['video_name'] == video_name]
            video_df = video_df.reset_index(drop=True)
            video_df = video_df.sort_values('frame_num')
            
            emo_pos_df['img_path'] = video_df['img_path']
            emo_pos_df['frame_num'] = video_df['frame_num']
            emo_pos_df[run_name] = video_df['emo_pos']
            
            base_video_df = base_df[base_df['video_name'] == video_name]
            base_video_df = base_video_df.reset_index(drop=True)
            base_video_df = base_video_df.sort_values('frame_num')
    
            
            # plot
            if legend_list is not None:
                fig.add_trace(go.Scatter(x=video_df['frame_num'], y=video_df['emo_pos'], mode='markers', marker=dict(color=color), name=legend_list[i], showlegend=False), row=row, col=col)
            else:
                # color is random
                fig.add_trace(go.Scatter(x=video_df['frame_num'], y=video_df['emo_pos'], mode='markers', marker=dict(color=color), name=run_name, showlegend=False), row=row, col=col)
        
        # draw emo_pos average
        if show_average:
            emo_pos_df['mean'] = emo_pos_df.iloc[:, 2:].mean(axis=1)
            fig.add_trace(go.Scatter(x=emo_pos_df['frame_num'], y=emo_pos_df['mean'], mode='markers', name='average', showlegend=False, marker=dict(color=color_list[(i+1) % len(color_list)])), row=row, col=col)
        
        emo = 0
        for i in base_video_df['frame_num']:
            _emo = base_video_df[base_video_df['frame_num'] == i]['emotion'].values[0]
            if emo != _emo or i == 0:
                # write vertical line
                if _emo == 0:
                    fig.add_trace(go.Scatter(x=[i]*2, y=[0, 1], mode='lines', line=dict(color='blue', width=2, dash='dash'), showlegend=False), row=row, col=col)
                elif _emo == 1:
                    fig.add_trace(go.Scatter(x=[i]*2, y=[0, 1], mode='lines', line=dict(color='red', width=2, dash='dash'), showlegend=False), row=row, col=col)
                elif _emo == 2:
                    fig.add_trace(go.Scatter(x=[i]*2, y=[0, 1], mode='lines', line=dict(color='green', width=2, dash='dash'), showlegend=False), row=row, col=col)
            emo = _emo
        
        # horizontal line at 0.5
        fig.add_trace(go.Scatter(x=base_video_df['frame_num'], y=[0.5]*len(base_video_df), mode='lines', name='0.5', line=dict(color='black', width=1, dash='dash'), showlegend=False), row=row, col=col)
        fig.update_yaxes(title='posteriors', range=[0, 1], row=row, col=col)
        fig.update_xaxes(title='frame', row=row, col=col)
        col += 1
    
    # show legend
    for i in range(len(run_name_list)):
        if legend_list is not None:
            fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color=color_list[i % len(color_list)]), name=legend_list[i]))
        else:
            fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color=color_list[i % len(color_list)]), name=run_name_list[i]))
    
    if show_average:
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color=color_list[(i+1) % len(color_list)]), name='average'))

    fig.update_layout(showlegend=True, legend=dict(orientation="h", font=dict(size=24)))
    
    # tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=30))
    
    fig.show()

In [8]:
com_video_name_list_path = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/comfort_nomixed_seq_video_name_list_wsize30-ssize5.csv'
com_video_name_list = pd.read_csv(com_video_name_list_path).columns.tolist()

dis_video_name_list_path = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/discomfort_video_name_list_wsize150-ssize3.csv'
dis_video_name_list = pd.read_csv(dis_video_name_list_path).columns.tolist()

In [25]:
# 2023/11/07 compare stream 
run_name_list = ['4_d_a', '4_d_g', '4_d_h', '4_d_agh', '4_d_agh_wsum']
compare_emo_pos_all_video(run_name_list, dis_video_name_list, figure_size=(3000, 1000), epoch=[10, 10, 5, 5, 5])

In [26]:
run_name_list = ['4_d_a', '4_d_g', '4_d_h']
compare_emo_pos_all_video(run_name_list, dis_video_name_list, figure_size=(3000, 1000), epoch=[10, 10, 5])

In [10]:
run_name_list = ['4_d_a_ws150-ss3', '4_d_g_ws150-ss3']
compare_emo_pos_all_video(run_name_list, dis_video_name_list, figure_size=(3000, 1000), epoch=[5, 5])